# Proyecto | Base de Datos de Big Data
# Proyecto: Particionamiento y Muestreo sobre datos de Lending Club

Este cuaderno tiene como objetivo aplicar técnicas de particionamiento y muestreo sobre un conjunto de datos de préstamos del portal Lending Club. Se identifican variables clave, se construyen combinaciones de particionamiento, y se extraen subconjuntos de datos representativos para etapas futuras de aprendizaje automático o análisis.


In [133]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when,mean, stddev, count, desc,to_date, year,round,lit
from collections import Counter, defaultdict

In [134]:
spark = SparkSession.builder.appName("LendingClubLoanData").getOrCreate()
spark

## 2. Carga de datos 

In [135]:
df = spark.read.option("header", True).option("inferSchema", True).csv("E:/Tareas/BigData/A3/loan.csv")
df.show(10)

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+-------------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+

## 3. Limpieza y creación de columnas útiles

In [136]:
total_rows = df.count()

porcentaje_nulos = df.select([(sum(when(col(c).isNull() | (col(c) == ''), 1).otherwise(0)) / total_rows).alias(c)for c in df.columns])
porcentaje_nulos_pd = porcentaje_nulos.toPandas().T
porcentaje_nulos_pd.columns = ["porcentaje_nulo"]
columnas_validas = porcentaje_nulos_pd[porcentaje_nulos_pd["porcentaje_nulo"] < 0.2].index.tolist()
df = df.select(*columnas_validas)
print(f"Columnas seleccionadas: {len(columnas_validas)}")

Columnas seleccionadas: 87


Seleccion de variables

In [137]:
columnas = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade','emp_length', 'home_ownership', 'annual_inc', 'purpose', 'loan_status']
df = df.select(columnas)
df.show(10)

+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|           purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|     2500| 36 months|   13.56|      84.92|    C| 10+ years|          RENT|     55000|debt_consolidation|    Current|
|    30000| 60 months|   18.94|     777.23|    D| 10+ years|      MORTGAGE|     90000|debt_consolidation|    Current|
|     5000| 36 months|   17.97|     180.69|    D|   6 years|      MORTGAGE|     59280|debt_consolidation|    Current|
|     4000| 36 months|   18.94|     146.51|    D| 10+ years|      MORTGAGE|     92000|debt_consolidation|    Current|
|    30000| 60 months|   16.14|     731.78|    C| 10+ years|      MORTGAGE|     57250|debt_consolidation|    Current|
|     5550| 36 months|   15.02|     192.45|    C| 10+ ye

In [138]:
df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- loan_status: string (nullable = true)



Agrupamos valores poco frecuentes en las columnas `purpose`,`home_ownership` y `loan_status` para reducir el número de combinaciones.

In [139]:
from pyspark.sql.functions import when


categorias_frecuentes = ['debt_consolidation', 'credit_card', 'home_improvement']

df = df.withColumn(
    "purpose",
    when(col("purpose").isin(categorias_frecuentes), col("purpose"))
    .otherwise("other")
)

categorias_frecuentes = ['MORTGAGE','RENT','OWN']

df = df.withColumn(
    "home_ownership",
    when(col("home_ownership").isin(categorias_frecuentes), col("home_ownership"))
    .otherwise("OTHER")
)
df = df.withColumn(
    "loan_status",
    when(col("loan_status").isin("Fully Paid", "Current"), "no_riesgo")
    .when(col("loan_status").isin("Charged Off", "Late (31-120 days)", "Late (16-30 days)"), "riesgo_medio")
    .otherwise("riesgo_alto")
)

Estadística descriptiva para las 10 variables seleccionadas 

In [140]:
df.describe('loan_amnt', 'int_rate', 'installment', 'annual_inc').show()

+-------+------------------+-----------------+------------------+-----------------+
|summary|         loan_amnt|         int_rate|       installment|       annual_inc|
+-------+------------------+-----------------+------------------+-----------------+
|  count|           2260668|          2260668|           2260668|          2260664|
|   mean|15046.931227849467|13.09291294431385|445.80764592588497|77992.44637764234|
| stddev| 9190.245488232782|4.832114232872253| 267.1737249936657|112696.2213609144|
|    min|               500|             5.31|              4.93|                0|
|    max|             40000|            30.99|           1719.83|         MORTGAGE|
+-------+------------------+-----------------+------------------+-----------------+



In [141]:
categoricas = ['term', 'grade', 'emp_length', 'home_ownership', 'purpose', 'loan_status']

for col_name in categoricas:
    print(f"\nDistribución de la variable '{col_name}':")
    
    total_count = df.select(col_name).count()
    
    df.groupBy(col_name) \
      .agg(count("*").alias("frecuencia")) \
      .withColumn("porcentaje (%)", round((col("frecuencia") / total_count) * 100, 2)) \
      .orderBy("frecuencia", ascending=False) \
      .show(truncate=False)



Distribución de la variable 'term':
+----------+----------+--------------+
|term      |frecuencia|porcentaje (%)|
+----------+----------+--------------+
| 36 months|1609754   |71.21         |
| 60 months|650914    |28.79         |
+----------+----------+--------------+


Distribución de la variable 'grade':
+-----+----------+--------------+
|grade|frecuencia|porcentaje (%)|
+-----+----------+--------------+
|B    |663557    |29.35         |
|C    |650053    |28.75         |
|A    |433027    |19.15         |
|D    |324424    |14.35         |
|E    |135639    |6.0           |
|F    |41800     |1.85          |
|G    |12168     |0.54          |
+-----+----------+--------------+


Distribución de la variable 'emp_length':
+----------+----------+--------------+
|emp_length|frecuencia|porcentaje (%)|
+----------+----------+--------------+
|10+ years |748005    |33.09         |
|2 years   |203676    |9.01          |
|< 1 year  |189988    |8.4           |
|3 years   |180753    |8.0           |

##  5. Particionamiento: combinaciones y probabilidades

In [142]:
from pyspark.sql.functions import col, round, count

total = df.count()

combinaciones = df.groupBy("loan_status", "home_ownership", "purpose") \
    .count() \
    .withColumnRenamed("count", "frecuencia") \
    .withColumn("probabilidad (%)", round((col("frecuencia") / total) * 100, 2)) \
    .orderBy("frecuencia", ascending=False)

print("Combinaciones posibles:", combinaciones.count())
combinaciones.show(truncate=False)


Combinaciones posibles: 48
+------------+--------------+------------------+----------+----------------+
|loan_status |home_ownership|purpose           |frecuencia|probabilidad (%)|
+------------+--------------+------------------+----------+----------------+
|no_riesgo   |MORTGAGE      |debt_consolidation|551048    |24.38           |
|no_riesgo   |RENT          |debt_consolidation|432505    |19.13           |
|no_riesgo   |MORTGAGE      |credit_card       |219769    |9.72            |
|no_riesgo   |RENT          |credit_card       |188961    |8.36            |
|no_riesgo   |RENT          |other             |125938    |5.57            |
|no_riesgo   |OWN           |debt_consolidation|111406    |4.93            |
|no_riesgo   |MORTGAGE      |other             |110795    |4.9             |
|no_riesgo   |MORTGAGE      |home_improvement  |101616    |4.49            |
|riesgo_medio|RENT          |debt_consolidation|82060     |3.63            |
|riesgo_medio|MORTGAGE      |debt_consolidation|7

 ## 6. Filtrado de una partición

In [143]:
df_part1 = df.filter(
    (col("loan_status") == "no_riesgo") &
    (col("home_ownership") == "MORTGAGE") &
    (col("purpose") == "debt_consolidation")
)

print("Cantidad de registros en la partición:", df_part1.count())
df_part1.show(5)


Cantidad de registros en la partición: 551048
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|           purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|    30000| 60 months|   18.94|     777.23|    D| 10+ years|      MORTGAGE|     90000|debt_consolidation|  no_riesgo|
|     5000| 36 months|   17.97|     180.69|    D|   6 years|      MORTGAGE|     59280|debt_consolidation|  no_riesgo|
|     4000| 36 months|   18.94|     146.51|    D| 10+ years|      MORTGAGE|     92000|debt_consolidation|  no_riesgo|
|    30000| 60 months|   16.14|     731.78|    C| 10+ years|      MORTGAGE|     57250|debt_consolidation|  no_riesgo|
|     5000| 36 months|   17.97|     180.69|    D| 10+ years|      MORTGAGE|     53580|debt_consolidation|  no_riesgo|
+---------

In [144]:
df_part2 = df.filter(
    (col("loan_status") == "no_riesgo") &
    (col("home_ownership") == "RENT") &
    (col("purpose") == "debt_consolidation")
)

print("Cantidad de registros en la partición:", df_part2.count())
df_part2.show(5)

Cantidad de registros en la partición: 432505
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|           purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|     2500| 36 months|   13.56|      84.92|    C| 10+ years|          RENT|     55000|debt_consolidation|  no_riesgo|
|     2000| 36 months|   17.97|      72.28|    D|   4 years|          RENT|     51000|debt_consolidation|  no_riesgo|
|    23000| 60 months|   20.89|     620.81|    D|   5 years|          RENT|     68107|debt_consolidation|  no_riesgo|
|    20000| 36 months|   12.98|     673.69|    B|   5 years|          RENT|     74800|debt_consolidation|  no_riesgo|
|     2200| 36 months|   15.02|      76.29|    C|       n/a|          RENT|     70000|debt_consolidation|  no_riesgo|
+---------

In [145]:
df_part3 = df.filter(
    (col("loan_status") == "no_riesgo") &
    (col("home_ownership") == "MORTGAGE") &
    (col("purpose") == "credit_card")
)

print("Cantidad de registros en la partición:", df_part3.count())
df_part3.show(5)

Cantidad de registros en la partición: 219769
+---------+----------+--------+-----------+-----+----------+--------------+----------+-----------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|    purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+-----------+-----------+
|     5550| 36 months|   15.02|     192.45|    C| 10+ years|      MORTGAGE|    152500|credit_card|  no_riesgo|
|     5500| 36 months|   22.35|     211.05|    D|  < 1 year|      MORTGAGE|     50000|credit_card|  no_riesgo|
|    28000| 60 months|   11.31|     613.13|    B|   2 years|      MORTGAGE|     70000|credit_card|  no_riesgo|
|    11200| 36 months|    8.19|     351.95|    A| 10+ years|      MORTGAGE|     65000|credit_card|  no_riesgo|
|    16000| 60 months|   20.89|     431.87|    D|   4 years|      MORTGAGE|    120000|credit_card|  no_riesgo|
+---------+----------+--------+-----------+-----+----------+------

In [146]:
df_part4 = df.filter(
    (col("loan_status") == "no_riesgo") &
    (col("home_ownership") == "RENT") &
    (col("purpose") == "credit_card")
)

print("Cantidad de registros en la partición:", df_part4.count())
df_part4.show(5)

Cantidad de registros en la partición: 188961
+---------+----------+--------+-----------+-----+----------+--------------+----------+-----------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|    purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+-----------+-----------+
|     6000| 36 months|   13.56|     203.79|    C| 10+ years|          RENT|     65000|credit_card|  no_riesgo|
|     9600| 36 months|    23.4|     373.62|    E|   9 years|          RENT|     65000|credit_card|  no_riesgo|
|    16000| 60 months|   26.31|     481.99|    E|  < 1 year|          RENT|     33000|credit_card|  no_riesgo|
|     3500| 36 months|   20.89|     131.67|    D| 10+ years|          RENT|     33190|credit_card|  no_riesgo|
|    10000| 36 months|   20.89|     376.19|    D|   7 years|          RENT|     74000|credit_card|  no_riesgo|
+---------+----------+--------+-----------+-----+----------+------

## 7. Muestreo aleatorio de la partición

Se extrae el 10% de los registros de forma aleatoria y sin reemplazo.

In [148]:
df_part1_sample = df_part1.sample(withReplacement=False, fraction=0.1, seed=42)
print("Cantidad en la muestra:", df_part1_sample.count())
df_part1_sample.show(5)

Cantidad en la muestra: 55225
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|loan_amnt|      term|int_rate|installment|grade|emp_length|home_ownership|annual_inc|           purpose|loan_status|
+---------+----------+--------+-----------+-----+----------+--------------+----------+------------------+-----------+
|     3500| 36 months|   16.14|      123.3|    C| 10+ years|      MORTGAGE|     80000|debt_consolidation|  no_riesgo|
|    40000| 60 months|   14.47|     940.51|    C|  < 1 year|      MORTGAGE|    500000|debt_consolidation|  no_riesgo|
|    26000| 60 months|   18.94|      673.6|    D| 10+ years|      MORTGAGE|     40000|debt_consolidation|  no_riesgo|
|    15000| 60 months|   14.47|     352.69|    C| 10+ years|      MORTGAGE|    120000|debt_consolidation|  no_riesgo|
|    35000| 60 months|   15.02|     833.02|    C| 10+ years|      MORTGAGE|    106095|debt_consolidation|  no_riesgo|
+---------+----------+----

## 8. Conclusión

## Conclusión

Este análisis permitió:
- Identificar combinaciones significativas en la base de datos.
- Extraer particiones específicas.
- Aplicar muestreo aleatorio para reducir volumen de datos de forma representativa.

